# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow([
        'artist',
        'firstName',
        'gender',
        'itemInSession',
        'lastName',
        'length',
        'level',
        'location',
        'sessionId',
        'song',
        'userId'
        ]
    )
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


#### Creating a Cluster

In [4]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [5]:
# Create a Keyspace 

try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS project
        WITH REPLICATION = {
        'class': 'SimpleStrategy', 
        'replication_factor':1
        }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [6]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace("project")
except Exception as e:
    print(e)

# Part II. Creating tables and QA the following queries.

### 1. Artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### A: So, during sessionId = 338 and itemInSession = 4, the song "Music Matters (Mark Knight Dub)" by the artist "Faithless" with a length of approximately 495 seconds was heard in the music app history.

In [7]:
# Define a CQL (Cassandra Query Language) query to create a table if it does not already exist.
query = """
    CREATE TABLE IF NOT EXISTS artist_songs (
        sessionId INT,
        itemInSession INT,
        artist TEXT,
        length FLOAT,
        song TEXT,
        PRIMARY KEY (sessionId, itemInSession, song)
    )
"""

# Attempt to execute the above query using a Cassandra session.
try:
    session.execute(query)
except Exception as e:
    # If an exception occurs during execution, print the error message.
    print(e)

In [ ]:
# Define the path to the CSV file to be read.
file = 'event_datafile_new.csv'

# Open the CSV file for reading with UTF-8 encoding.
with open(file, encoding='utf8') as f:
    # Create a CSV reader object.
    csvreader = csv.reader(f)
    
    # Skip the header row (the first row) of the CSV file.
    next(csvreader)
    
    # Iterate through each line in the CSV file.
    for line in csvreader:
        # Define an initial CQL query for inserting data into the Cassandra table.
        query = """
            INSERT INTO artist_songs (
                sessionId,
                itemInSession,
                artist,
                length,
                song
            )
        """
        
        # Add the "VALUES" clause to the query and specify placeholders for values.
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        # Extract values from the current CSV line and execute the query with those values.
        session.execute(query, (int(line[8]), int(line[3]), line[0], float(line[5]), line[9]))


In [ ]:
# Define a CQL (Cassandra Query Language) query to retrieve data from the Cassandra table.
query = """
    SELECT
        artist,
        length,
        song
    FROM artist_songs
    WHERE
        sessionId = 338
        AND itemInSession = 4
"""

# Attempt to execute the query using a Cassandra session.
try:
    # Execute the query and store the result in the "rows" variable.
    rows = session.execute(query)
except Exception as e:
    # If an exception occurs during execution, print the error message.
    print(e)

# Iterate through the rows of the result set and print each row.
for row in rows:
    print(row)

### 2. Name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### A: For userid = 10 and sessionid = 182, the following songs were played, and their details are sorted by iteminsession:

Song Title: Keep On Keepin' On

* Artist: Down To The Bone
* User: Sylvie Cruz (first name: Sylvie, last name: Cruz)  
* Item in Session: 0  

Song Title: Greece 2000

* Artist: Three Drives
* User: Sylvie Cruz (first name: Sylvie, last name: Cruz)  
* Item in Session: 1  

Song Title: Kilometer  

* Artist: Sebastien Tellier
* User: Sylvie Cruz (first name: Sylvie, last name: Cruz)  
* Item in Session: 2  

Song Title: Catch You Baby (Steve Pitron & Max Sanna Radio Edit)

* Artist: Lonnie Gordon  
* User: Sylvie Cruz (first name: Sylvie, last name: Cruz)  
* Item in Session: 3  

In [ ]:
# Define a CQL (Cassandra Query Language) query to create a table if it does not already exist.
query = """
    CREATE TABLE IF NOT EXISTS user_session_songs (
        userId INT,
        sessionId INT,
        itemInSession INT,
        artist TEXT,
        song TEXT,
        firstName TEXT,
        lastName TEXT,
        PRIMARY KEY (userId, sessionId, itemInSession)
    ) WITH CLUSTERING ORDER BY (sessionId ASC, itemInSession ASC)
"""

# Attempt to execute the above query using a Cassandra session.
try:
    session.execute(query)
except Exception as e:
    # If an exception occurs during execution, print the error message.
    print(e)

In [ ]:
# Define the path to the CSV file to be read.
file = 'event_datafile_new.csv'

# Open the CSV file for reading with UTF-8 encoding.
with open(file, encoding='utf8') as f:
    # Create a CSV reader object.
    csvreader = csv.reader(f)
    
    # Skip the header row (the first row) of the CSV file.
    next(csvreader)
    
    # Iterate through each line of the CSV file.
    for line in csvreader:
        # Define an initial CQL query for inserting data into the Cassandra table.
        query = """
            INSERT INTO user_session_songs (
                userId,
                sessionId,
                itemInSession,
                artist,
                song,
                firstName,
                lastName
            )
        """
        
        # Add the "VALUES" clause to the query and specify placeholders for values.
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        # Extract values from the current CSV line and execute the query with those values.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [ ]:
# Define a CQL (Cassandra Query Language) query to retrieve data from the Cassandra table.
query = """
    SELECT
        itemInSession,
        artist,
        song,
        firstName,
        lastName
    FROM user_session_songs
    WHERE
        userId = 10
        AND sessionId = 182
"""

# Attempt to execute the query using a Cassandra session.
try:
    # Execute the query and store the result in the "rows" variable.
    rows = session.execute(query)
except Exception as e:
    # If an exception occurs during execution, print the error message.
    print(e)

# Iterate through the rows of the result set and print each row.
for row in rows:
    print(row)

### 3. Every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### A: The following users listened to the specified song:
* Jacqueline Lynch
* Sara Johnson
* Tegan Levine

In [ ]:
# Define a CQL (Cassandra Query Language) query to create a table if it does not already exist.
query = """
    CREATE TABLE IF NOT EXISTS song_listeners (
        song TEXT,
        userId INT,
        firstName TEXT,
        lastName TEXT,
        PRIMARY KEY (song, userId)
    )
"""

# Attempt to execute the above query using a Cassandra session.
try:
    session.execute(query)
except Exception as e:
    # If an exception occurs during execution, print the error message.
    print(e)

In [ ]:
# Define the path to the CSV file to be read.
file = 'event_datafile_new.csv'

# Open the CSV file for reading with UTF-8 encoding.
with open(file, encoding='utf8') as f:
    # Create a CSV reader object.
    csvreader = csv.reader(f)
    
    # Skip the header row (the first row) of the CSV file.
    next(csvreader)
    
    # Iterate through each line in the CSV file.
    for line in csvreader:
        # Define an initial CQL query for inserting data into the Cassandra table.
        query = """
            INSERT INTO song_listeners (
                song,
                userId,
                firstName,
                lastName
            )
        """
        
        # Add the "VALUES" clause to the query and specify placeholders for values.
        query = query + "VALUES (%s, %s, %s, %s)"
        
        # Extract values from the current CSV line and execute the query with those values.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))


In [ ]:
# Define a CQL (Cassandra Query Language) query to retrieve data from the Cassandra table.
query = """
    SELECT
        userId,
        firstName,
        lastName
    FROM song_listeners
    WHERE
        song = 'All Hands Against His Own'
"""

# Attempt to execute the query using a Cassandra session.
try:
    # Execute the query and store the result in the "rows" variable.
    rows = session.execute(query)
except Exception as e:
    # If an exception occurs during execution, print the error message.
    print(e)

# Iterate through the rows of the result set and print each row.
for row in rows:
    print(row)


### Drop the tables before closing out the sessions

In [ ]:
tables = ["artist_songs", "user_session_songs", "song_listeners"]


for tb in tables:
    try:
        rows = session.execute("DROP TABLE "+tb)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()